In [ ]:
import random
import numpy as np
from copy import copy, deepcopy
from tqdm import tqdm
from itertools import permutations, takewhile

In [ ]:
def read_instance(filename):

    with open(filename, 'r') as f:
        lines = f.readlines()

    first_line = lines[0].split()
    n = int(first_line[0])
    m = int(first_line[1])

    processing_times = [[0]*m for _ in range(n)]
    for i in range(n):
        line = lines[i+1].split()
        for j in range(m):
            processing_times[i][j] = int(line[j])
    
    last_line = lines[-1].split()
    ub = int(last_line[0])
    lb = int(last_line[1])

    return n, m, processing_times, ub, lb


In [ ]:
class Job():

    def __init__(self, job_id, processing_times):
        self.id = job_id
        self.processing_times = processing_times
        self.time_available = 0
        self.remaining_time = sum(processing_times)
        self.remaining_machines = set([i for i in range(len(processing_times))])
        
class Machine():
    def __init__(self, machine_id, processing_times):
        self.id = machine_id
        self.time_available = 0
        self.remaining_time = sum([processing_times[i][machine_id] for i in range(len(processing_times[0]))])
        self.remaining_jobs = set([i for i in range(len(processing_times[0]))])

def update(machine, job, processing_times):
    duration = processing_times[job.id][machine.id]
    end  = max(machine.time_available, job.time_available) + duration
    machine.remaining_time -= duration
    machine.remaining_jobs.remove(job.id)
    machine.time_available = end
    job.remaining_time -= duration
    job.remaining_machines.remove(machine.id)
    job.time_available = end

In [ ]:
from itertools import takewhile


def makespan(scheduling, processing_times):
    n = len(scheduling[0])
    m = len(scheduling)
    permutation=[]

    jobs = [Job(i, processing_times[i]) for i in range(n)]
    machines = [Machine(j, processing_times) for j in range(m)]

    for j in range(n):
        machine_candidates = list(takewhile(lambda m: m.time_available == machines[0].time_available, machines))
        if len(machine_candidates)>1:
            machine_candidates.sort(key = lambda m: m.remaining_time, reverse=True)
            machine_candidates = list(takewhile(lambda m: m.remaining_time == machine_candidates[0].remaining_time, machine_candidates))
            if len(machine_candidates)>1:
                job_candidates = [-1 for _ in range(m)]
                for machine in machine_candidates:
                    job_candidates[machine.id] = scheduling[machine.id][j]
                machine_candidates.sort(key = lambda m: jobs[job_candidates[m.id]].remaining_time - processing_times[job_candidates[m.id]][m.id], reverse=True)
        candidates_ids= [machine.id for machine in machine_candidates]
        machines = machine_candidates + [machine for machine in machines if machine.id not in candidates_ids]
        for machine in machines:
            permutation.append(jobs[scheduling[machine.id][j]].id*n+machine.id)
            update(machine, jobs[scheduling[machine.id][j]], processing_times)
            
        machines.sort(key = lambda m: m.time_available)
       
    return (np.max([machine.time_available for machine in machines]), permutation)


In [ ]:
from copy import copy

def scheduling(processing_times):
    n = len(processing_times)
    m = len(processing_times[0])
    schedule = [[0]*n for _ in range(m)]
    
    jobs = [Job(i, processing_times[i]) for i in range(n)]
    machines = [Machine(j, processing_times) for j in range(m)]
    
    next_machine_candidates = copy(machines)
    next_machine_candidates.sort(key = lambda m: m.remaining_time, reverse=True)
    first_machine = next_machine_candidates[0]
    job_candidates = copy(jobs)
    job_candidates.sort(key = lambda job: job.remaining_time - processing_times[job.id][first_machine.id], reverse=True)
    first_job = job_candidates[0]
    schedule[first_machine.id][0] = first_job.id
    update(first_machine, first_job, processing_times)
    
    for k in range(1, n*m):

        next_machine_candidates.sort(key = lambda m: m.time_available)
       
        next_job_candidates = []
        for next_machine in next_machine_candidates:
            job_candidates = []
            for job in next_machine.remaining_jobs:
                if jobs[job].time_available<=next_machine.time_available:
                    job_candidates.append((jobs[job],jobs[job].remaining_time-jobs[job].processing_times[next_machine.id]))
            if len(job_candidates)!=0:
                next_machine_id = next_machine.id
                next_job_candidates = copy(job_candidates)
                break
        if len(next_job_candidates)==0:
            for machine in next_machine_candidates: 
                if len(machine.remaining_jobs)!=0:
                    next_machine_id=machine.id
                    break
            remaining_jobs = list(machines[next_machine_id].remaining_jobs)
            remaining_jobs.sort(key = lambda job: jobs[job].remaining_time-jobs[job].processing_times[next_machine_id], reverse=True)
            job = remaining_jobs[0]
            next_job_candidates.append((jobs[job], jobs[job].remaining_time-jobs[job].processing_times[next_machine_id]))
    
        else:
            next_job_candidates.sort(key = lambda job: job[1], reverse=True)

        next_job = next_job_candidates[0][0]
        next_machine = machines[next_machine_id]
        schedule[next_machine.id][len(processing_times)-len(next_machine.remaining_jobs)] = next_job.id
        update(next_machine, next_job, processing_times)
        
    return schedule

In [ ]:
def swap_operations(schedule):
    neighbor = deepcopy(schedule)
    row = random.choice(range(len(schedule)))
    col1, col2 = random.sample(range(len(schedule[0])),2)
    schedule[row][col1], schedule[row][col2] = schedule[row][col2], schedule[row][col1]
    
    return schedule

def simulated_annealing(n, m, processing_times, max_iters):
    solution = scheduling(processing_times)
    value = makespan(solution,processing_times)[0]
    best_solution = deepcopy(solution)
    best_value = value
    
    for i in (range(1, max_iters)):

        new_solution = swap_operations(solution)
        
        new_value = makespan(new_solution, processing_times)[0]
        
        if new_value < value:
            value = new_value
            solution = new_solution
            if new_value < best_value:
                best_value = new_value
                best_solution = deepcopy(new_solution)
        else:
            p = 1 / i**0.5 
            q = random.random()
            if q < p:
                value = new_value
                solution = new_solution
                
    return makespan(best_solution, processing_times)



In [ ]:
bounds = [(0,0) for _ in range(10)]
results = [[] for _ in range(10)]
for i in range(10):
    instance_file = "tests/test55" + str(i)
    n, m, processing_times, ub, lb = read_instance(instance_file)
    bounds[i] = (lb, ub)
    j=0
    for it in tqdm(range(100)):
        results[i].append(simulated_annealing(n, m, processing_times, 100000))
        j+=1
    print(min(results[i]))

In [ ]:
print('lower bounds: ')
print([bounds[i][0] for i in range(10)])
print('upper_bounds: ')
print([bounds[i][1] for i in range(10)])

best = [min([results[i][j][0] for j in range(len(results[i]))]) for i in range(10)]
mean = [int(sum([results[i][j][0] for j in range(len(results[i]))])/(len(results[i]))) for i in range(10)]
print('best: ')
print(best)
print('mean')
print(mean)

In [ ]:
from matplotlib import pyplot as plt
plt.plot(range(10), [bounds[i][0] for i in range(10)])
plt.plot(range(10), [bounds[i][1] for i in range(10)])
plt.plot(range(10), best)
plt.show()
print([bounds[i][1] for i in range(10)])
print( best)

In [ ]:
processing_times= [[[52,13,77], [11,20,42], [7, 44, 32]], 
                   [[40, 5,18], [30,21,14], [8, 37, 16]],
                   [[15,31,72], [19,30,62], [22, 54, 12]],
                   [[19,30,62], [52,13,77], [19, 34, 28]],
                   [[27,32,51], [11,40,20], [71, 40, 60]],
                   [[20,37,77], [61,10,42], [70, 15, 43]],
                   [[12,51,17], [41,20,12], [16, 50, 29]],
                   [[50,10,70], [10,20,40], [70, 40, 30]],
                   [[29,16,67], [43,50,17], [63, 4, 72]],
                   [[14,25,69], [17,50,22], [81, 12, 72]]
    
]

In [ ]:
def brute_force(processing_times):

    perms = list(permutations(range(3)))
    min_makespan = float('inf')

    p = len(perms)

    #3 machines
    for i in range(p):
        for j in range(p):
            for k in range(p):
                solution = []
                solution.append(perms[i])
                solution.append(perms[j])
                solution.append(perms[k])

                time = makespan(solution,processing_times)[0]
                if time < min_makespan:
                    min_makespan = time

    return min_makespan


In [ ]:
brute_force_results=[]
results=[]
for times in processing_times:
    brute_force_results.append(brute_force(times))
    results.append(simulated_annealing(3, 3, times, 1000))
    
print(brute_force_results)
print([res[0] for res in results])